In [3]:
from __future__ import print_function
import keras
from sklearn.utils import shuffle
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Input, GlobalMaxPooling2D, BatchNormalization
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint,EarlyStopping
import tensorflow as tf
from keras import backend as K

from sklearn import metrics
from sklearn.metrics import auc,precision_recall_curve,recall_score, roc_auc_score
import rdkit
from rdkit import Chem
from rdkit.Chem import AllChem
import pandas as pd
import numpy as np
#from matplotlib import pyplot as plt
#%matplotlib inline
print("RDKit: %s"%rdkit.__version__)
print("Keras: %s"%keras.__version__)

from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten
from keras.layers import Embedding, LSTM, GlobalMaxPooling1D, SpatialDropout1D, Conv1D, MaxPooling1D, GRU, Bidirectional


RDKit: 2020.03.3
Keras: 2.4.3


### 以下是一些rnn的预处理函数

In [4]:
def vectorize(smiles):
        one_hot =  np.zeros((smiles.shape[0], embed , len(charset)),dtype=np.int8)
        for i,smile in enumerate(smiles):
            #encode the startchar
            one_hot[i,0,char_to_int["!"]] = 1
            #encode the rest of the chars
            for j,c in enumerate(smile):
                one_hot[i,j+1,char_to_int[c]] = 1
            #Encode endchar
            one_hot[i,len(smile)+1:,char_to_int["E"]] = 1
        #Return two, one for input and the other for output
        return one_hot[:,0:-1,:], one_hot[:,1:,:]

def coeff_determination(y_true, y_pred):
    from keras import backend as K
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )
 
def get_lr_metric(optimizer):
    def lr(y_true, y_pred):
        return optimizer.lr
    return lr

def auc_roc(y_true, y_pred):
    # any tensorflow metric
    value, update_op = tf.contrib.metrics.streaming_auc(y_pred, y_true)

    # find all variables created for this metric
    metric_vars = [i for i in tf.local_variables() if 'auc_roc' in i.name.split('/')[1]]

    # Add metric variables to GLOBAL_VARIABLES collection.
    # They will be initialized for new session.
    for v in metric_vars:
        tf.add_to_collection(tf.GraphKeys.GLOBAL_VARIABLES, v)

    # force to update metric values
    with tf.control_dependencies([update_op]):
        value = tf.identity(value)
        return value

def auc_roc(y_true, y_pred):
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred, pos_label=1)
    return metrics.auc(fpr, tpr) 

def auc_prc(y_true,y_pred):
    precision, recall, thresholds=metrics.precision_recall_curve(y_true, y_pred, pos_label=1)
    return metrics.auc(precision, recall)

In [5]:
    roc1=[]
    prc1=[]
    roc2=[]
    prc2=[]
    roc3=[]
    prc3=[]
    roc4=[]
    prc4=[]

In [6]:
%cd ./pseudomonas/train_cv

/home/mist/pseudomonas/train_cv


## 第一个网络结构的实验，采用两层双向lstm层

In [20]:
for i in range(10):
    data = pd.read_csv('fold_'+str(1)+'/train.csv')
    datall= pd.read_csv("trainall.csv")
    test=pd.read_csv('fold_'+str(1)+'/test.csv')
    X_train_smiles = np.array(list(data["smiles"]))
    X_test_smiles= np.array(list(test["smiles"]))

    assay = "activity"  
    Y_train = data[assay].values.reshape(-1,1)
    Y_test = test[assay].values.reshape(-1,1)
    charset = set("".join(list(datall.smiles))+"!E")
    char_to_int = dict((c,i) for i,c in enumerate(charset))
    int_to_char = dict((i,c) for i,c in enumerate(charset))
    embed = max([len(smile) for smile in datall.smiles]) + 5

    X_train, _ = vectorize(X_train_smiles)
    X_test, _ = vectorize(X_test_smiles)
    mol_str_train=[]
    mol_str_test=[]
    for x in range(len(data)):

        mol_str_train.append("".join([int_to_char[idx] for idx in np.argmax(X_train[x,:,:], axis=1)]))
    for x in range(len(test)):
        mol_str_test.append("".join([int_to_char[idx] for idx in np.argmax(X_test[x,:,:], axis=1)]))
    vocab_size=len(charset)
    mol_str_train = np.asarray(mol_str_train)
    mol_str_test = np.asarray(mol_str_test)
    vocab_size=len(charset)
    K.clear_session()
    # 网络结构一    
    model1= Sequential()
    model1.add(Embedding(vocab_size, 50, input_length=embed-1))
    model1.add(Bidirectional(LSTM(128, return_sequences=True)))
    model1.add(Bidirectional(LSTM(128)))
    model1.add(Dense(128, activation='relu'))
    model1.add(Dropout(0.4))
    model1.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(lr=0.001)
    lr_metric = get_lr_metric(optimizer)
    
    model1.compile(loss="mse", optimizer=optimizer, metrics=[coeff_determination, lr_metric])

    callbacks_list = [
    ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=1e-15, verbose=1, mode='auto',cooldown=0),
    ModelCheckpoint(filepath="weights.best.hdf5", monitor='val_loss', save_best_only=True, verbose=1, mode='auto'),
    EarlyStopping(monitor='loss', min_delta=0.001, patience=3, verbose=1, mode='auto', baseline=None, restore_best_weights=False)
    ]



    history =model1.fit(x=np.argmax(X_train, axis=2), y=Y_train,
                              batch_size=512,
                              epochs=10,
                              callbacks=callbacks_list)


    ynew = model1.predict(x=np.argmax(X_test, axis=2))
    roc1.append(auc_roc(Y_test, ynew))
    fpr1, tpr1, thresholds=(precision_recall_curve(Y_test,ynew))
    prc1.append(metrics.auc( tpr1,fpr1))
        #prc.append(auc_prc(Y_test, ynew))
    hist = history.history

Epoch 1/10
4/4 [==============================] - 17s 4s/step - loss: 0.2331 - coeff_determination: -13.0155 - lr: 0.0010
Epoch 2/10
4/4 [==============================] - 16s 4s/step - loss: 0.0960 - coeff_determination: -5.1650 - lr: 0.0010
Epoch 3/10
4/4 [==============================] - 17s 4s/step - loss: 0.0176 - coeff_determination: -0.0142 - lr: 0.0010
Epoch 4/10
4/4 [==============================] - 17s 4s/step - loss: 0.0177 - coeff_determination: -0.0179 - lr: 0.0010
Epoch 5/10
4/4 [==============================] - 16s 4s/step - loss: 0.0177 - coeff_determination: -0.0171 - lr: 0.0010
Epoch 6/10
4/4 [==============================] - 17s 4s/step - loss: 0.0177 - coeff_determination: -0.0183 - lr: 0.0010
Epoch 00006: early stopping
Epoch 1/10
4/4 [==============================] - 17s 4s/step - loss: 0.2162 - coeff_determination: -14.1712 - lr: 0.0010
Epoch 2/10
4/4 [==============================] - 17s 4s/step - loss: 0.0425 - coeff_determination: -1.0486 - lr: 0.0010
Ep

In [21]:
print(roc1)
print(prc1)
print(np.mean(roc1))
print(np.mean(prc1))

[0.7536094224924011, 0.40919452887537994, 0.297112462006079, 0.8301671732522796, 0.7237841945288754, 0.5277355623100304, 0.6369680851063829, 0.7120060790273557, 0.4242021276595745, 0.6685030395136778, 0.5776975683890578, 0.5330547112462006, 0.5112082066869301, 0.564209726443769, 0.7323328267477204]
[0.06651012525037528, 0.08775580648439259, 0.1043036422060421, 0.0548911653151144, 0.09504944433065954, 0.07305452597742822, 0.06594653770191838, 0.06768554352918923, 0.07153909934130863, 0.11509582612362489]
0.5934523809523811
0.08018317162600533


## 网络结构二

In [7]:
 for i in range(10):
    data = pd.read_csv('fold_'+str(1)+'/train.csv')
    datall= pd.read_csv("trainall.csv")
    test=pd.read_csv('fold_'+str(1)+'/test.csv')
    X_train_smiles = np.array(list(data["smiles"]))
    X_test_smiles= np.array(list(test["smiles"]))

    assay = "activity"  
    Y_train = data[assay].values.reshape(-1,1)
    Y_test = test[assay].values.reshape(-1,1)
    charset = set("".join(list(datall.smiles))+"!E")
    char_to_int = dict((c,i) for i,c in enumerate(charset))
    int_to_char = dict((i,c) for i,c in enumerate(charset))
    embed = max([len(smile) for smile in datall.smiles]) + 5

    X_train, _ = vectorize(X_train_smiles)
    X_test, _ = vectorize(X_test_smiles)
    mol_str_train=[]
    mol_str_test=[]
    for x in range(len(data)):

        mol_str_train.append("".join([int_to_char[idx] for idx in np.argmax(X_train[x,:,:], axis=1)]))
    for x in range(len(test)):
        mol_str_test.append("".join([int_to_char[idx] for idx in np.argmax(X_test[x,:,:], axis=1)]))
    vocab_size=len(charset)
    mol_str_train = np.asarray(mol_str_train)
    mol_str_test = np.asarray(mol_str_test)
    vocab_size=len(charset)
    K.clear_session()   
    # 网络结构二
    model2= Sequential()
    model2.add(Embedding(vocab_size, 50, input_length=embed-1))
    model2.add(Bidirectional(GRU(128, return_sequences=True)))
    model2.add(Bidirectional(GRU(128)))
    model2.add(Dense(128, activation='relu'))
    model2.add(Dropout(0.4))
    model2.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(lr=0.001)
    lr_metric = get_lr_metric(optimizer)
    
    model2.compile(loss="mse", optimizer=optimizer, metrics=[coeff_determination, lr_metric])

    callbacks_list = [
    ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=1e-15, verbose=1, mode='auto',cooldown=0),
    ModelCheckpoint(filepath="weights.best.hdf5", monitor='val_loss', save_best_only=True, verbose=1, mode='auto'),
    EarlyStopping(monitor='loss', min_delta=0.001, patience=3, verbose=1, mode='auto', baseline=None, restore_best_weights=False)
    ]



    history =model2.fit(x=np.argmax(X_train, axis=2), y=Y_train,
                              batch_size=512,
                              epochs=10,
                              callbacks=callbacks_list)


    ynew = model2.predict(x=np.argmax(X_test, axis=2))
    roc2.append(auc_roc(Y_test, ynew))
    fpr2, tpr2, thresholds=(precision_recall_curve(Y_test,ynew))
    prc2.append(metrics.auc( tpr2,fpr2))
    hist = history.history

Epoch 1/10
4/4 [==============================] - 15s 4s/step - loss: 0.2184 - coeff_determination: -10.9105 - lr: 0.0010
Epoch 2/10
4/4 [==============================] - 15s 4s/step - loss: 0.0889 - coeff_determination: -3.8413 - lr: 0.0010
Epoch 3/10
4/4 [==============================] - 16s 4s/step - loss: 0.0185 - coeff_determination: -0.0800 - lr: 0.0010
Epoch 4/10
4/4 [==============================] - 16s 4s/step - loss: 0.0177 - coeff_determination: -0.0169 - lr: 0.0010
Epoch 5/10
4/4 [==============================] - 15s 4s/step - loss: 0.0177 - coeff_determination: -0.0171 - lr: 0.0010
Epoch 6/10
4/4 [==============================] - 16s 4s/step - loss: 0.0177 - coeff_determination: -0.0194 - lr: 0.0010
Epoch 00006: early stopping
Epoch 1/10
4/4 [==============================] - 16s 4s/step - loss: 0.2274 - coeff_determination: -12.8770 - lr: 0.0010
Epoch 2/10
4/4 [==============================] - 16s 4s/step - loss: 0.1142 - coeff_determination: -5.8292 - lr: 0.0010
Ep

In [8]:
print(np.mean(roc2))
print(np.mean(prc2))

0.5855623100303952
0.08108724637037433


## 网络结构三

In [31]:
### 网络结构三

   for i in range(10):
    data = pd.read_csv('fold_'+str(1)+'/train.csv')
    datall= pd.read_csv("trainall.csv")
    test=pd.read_csv('fold_'+str(1)+'/test.csv')
    X_train_smiles = np.array(list(data["smiles"]))
    X_test_smiles= np.array(list(test["smiles"]))

    assay = "activity"  
    Y_train = data[assay].values.reshape(-1,1)
    Y_test = test[assay].values.reshape(-1,1)
    charset = set("".join(list(datall.smiles))+"!E")
    char_to_int = dict((c,i) for i,c in enumerate(charset))
    int_to_char = dict((i,c) for i,c in enumerate(charset))
    embed = max([len(smile) for smile in datall.smiles]) + 5

    X_train, _ = vectorize(X_train_smiles)
    X_test, _ = vectorize(X_test_smiles)
    mol_str_train=[]
    mol_str_test=[]
    for x in range(len(data)):

        mol_str_train.append("".join([int_to_char[idx] for idx in np.argmax(X_train[x,:,:], axis=1)]))
    for x in range(len(test)):
        mol_str_test.append("".join([int_to_char[idx] for idx in np.argmax(X_test[x,:,:], axis=1)]))
    vocab_size=len(charset)
    mol_str_train = np.asarray(mol_str_train)
    mol_str_test = np.asarray(mol_str_test)
    vocab_size=len(charset)
    K.clear_session()   
    # 网络结构三
    model3= Sequential()
    model3.add(Embedding(vocab_size, 50, input_length=embed-1))
    model3.add(Conv1D(192,3,activation='relu'))
    model3.add(Bidirectional(GRU(224, return_sequences=True)))
    model3.add(Bidirectional(GRU(384)))
    model3.add(Dense(128, activation='relu'))
    model3.add(Dropout(0.4))
    model3.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(lr=0.001)
    lr_metric = get_lr_metric(optimizer)
    
    model3.compile(loss="mse", optimizer=optimizer, metrics=[coeff_determination, lr_metric])

    callbacks_list = [
    ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=1e-15, verbose=1, mode='auto',cooldown=0),
    ModelCheckpoint(filepath="weights.best.hdf5", monitor='val_loss', save_best_only=True, verbose=1, mode='auto'),
    EarlyStopping(monitor='loss', min_delta=0.001, patience=3, verbose=1, mode='auto', baseline=None, restore_best_weights=False)
    ]



    history =model3.fit(x=np.argmax(X_train, axis=2), y=Y_train,
                              batch_size=512,
                              epochs=10,
                              callbacks=callbacks_list)

    ynew = model3.predict(x=np.argmax(X_test, axis=2))
    roc3.append(auc_roc(Y_test, ynew))
    fpr3, tpr3, thresholds=(precision_recall_curve(Y_test,ynew))
    prc3.append(metrics.auc( tpr3,fpr3))
    hist = history.history

In [12]:
print(np.mean(roc3))
print(np.mean(prc3))

0.6234647689780345
0.1113914354584879


## 网络结构四

In [ ]:
for i in range(10):
    data = pd.read_csv('fold_'+str(1)+'/train.csv')
    datall= pd.read_csv("trainall.csv")
    test=pd.read_csv('fold_'+str(1)+'/test.csv')
    X_train_smiles = np.array(list(data["smiles"]))
    X_test_smiles= np.array(list(test["smiles"]))

    assay = "activity"  
    Y_train = data[assay].values.reshape(-1,1)
    Y_test = test[assay].values.reshape(-1,1)
    charset = set("".join(list(datall.smiles))+"!E")
    char_to_int = dict((c,i) for i,c in enumerate(charset))
    int_to_char = dict((i,c) for i,c in enumerate(charset))
    embed = max([len(smile) for smile in datall.smiles]) + 5

    X_train, _ = vectorize(X_train_smiles)
    X_test, _ = vectorize(X_test_smiles)
    mol_str_train=[]
    mol_str_test=[]
    for x in range(len(data)):

        mol_str_train.append("".join([int_to_char[idx] for idx in np.argmax(X_train[x,:,:], axis=1)]))
    for x in range(len(test)):
        mol_str_test.append("".join([int_to_char[idx] for idx in np.argmax(X_test[x,:,:], axis=1)]))
    vocab_size=len(charset)
    mol_str_train = np.asarray(mol_str_train)
    mol_str_test = np.asarray(mol_str_test)
    vocab_size=len(charset)
    K.clear_session()   
    model = Sequential()
    model.add(Embedding(vocab_size, 50, input_length=embed-1))
    model.add(Conv1D(192, 10, activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv1D(192, 3, activation='relu'))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(lr=0.001)
    lr_metric = get_lr_metric(optimizer)
    
    
    model.compile(loss="mse", optimizer=optimizer, metrics=[coeff_determination, lr_metric])

    callbacks_list = [
    ReduceLROnPlateau(monitor='loss', factor=0.5, patience=5, min_lr=1e-15, verbose=1, mode='auto',cooldown=0),
    ModelCheckpoint(filepath="weights.best.hdf5", monitor='val_loss', save_best_only=True, verbose=1, mode='auto'),
    EarlyStopping(monitor='loss', min_delta=0.001, patience=3, verbose=1, mode='auto', baseline=None, restore_best_weights=False)
    ]



    history =model.fit(x=np.argmax(X_train, axis=2), y=Y_train,
                              batch_size=512,
                              epochs=10,
                              callbacks=callbacks_list)


    ynew = model.predict(x=np.argmax(X_test, axis=2))
    roc.append(auc_roc(Y_test, ynew))
        #prc.append(auc_prc(Y_test, ynew))
    hist = history.history

Epoch 1/10
4/4 [==============================] - 5s 1s/step - loss: 0.1031 - coeff_determination: -3.7061 - lr: 0.0010
Epoch 2/10
4/4 [==============================] - 5s 1s/step - loss: 0.0177 - coeff_determination: -0.0171 - lr: 0.0010
Epoch 3/10
4/4 [==============================] - 5s 1s/step - loss: 0.0177 - coeff_determination: -0.0206 - lr: 0.0010
Epoch 4/10
4/4 [==============================] - 5s 1s/step - loss: 0.0177 - coeff_determination: -0.0161 - lr: 0.0010
Epoch 5/10
4/4 [==============================] - 5s 1s/step - loss: 0.0177 - coeff_determination: -0.0195 - lr: 0.0010
Epoch 00005: early stopping
Epoch 1/10
4/4 [==============================] - 5s 1s/step - loss: 0.0876 - coeff_determination: -3.3649 - lr: 0.0010
Epoch 2/10
4/4 [==============================] - 5s 1s/step - loss: 0.0177 - coeff_determination: -0.0171 - lr: 0.0010
Epoch 3/10
4/4 [==============================] - 5s 1s/step - loss: 0.0177 - coeff_determination: -0.0183 - lr: 0.0010
Epoch 4/10
4

In [36]:
print(roc)
print(prc)



0.643473889
0.238439532
